<a href="https://colab.research.google.com/github/zeinabmoawad/Semantic-Search-Engine/blob/main/indexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from worst_case_implementation import VecDBWorst
import time
from dataclasses import dataclass
from typing import List
import os
import faiss


In [6]:
import os  
# delete codes file
for i in range(1000000):
    if os.path.exists("codes_"+str(i)+".txt"):
        os.remove("codes_"+str(i)+".txt")
for i in range(1000000):
    if os.path.exists("ivf_cluster_"+str(i)+".txt"):
        os.remove("ivf_cluster_"+str(i)+".txt")
    else:
        break

In [2]:
AVG_OVERX_ROWS = 10

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]

def run_queries(db, np_rows, top_k, num_runs):
    results = []
    for _ in range(num_runs):
        query = np.random.random((1,70))
        
        tic = time.time()
        db_ids = db.retrive(query, top_k)
        # print("db_ids = ",db_ids)
        toc = time.time()
        run_time = toc - tic
        tic = time.time()
        l2_index = faiss.IndexFlatL2(70)
        l2_index.add(np_rows)
        l2_dist, actual_ids = l2_index.search(query,len(np_rows))
        # actual_ids = np.argsort(np_rows.dot(query.T).T / (np.linalg.norm(np_rows, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()
        actual_ids = np.argsort(np_rows.dot(query.T).T / (np.linalg.norm(np_rows, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]
        # print("actual_ids = ",actual_ids[:5])
        toc = time.time()
        np_run_time = toc - tic
        
        actual_ids = np.ravel(actual_ids).tolist()
        results.append(Result(run_time, top_k, db_ids, actual_ids))
    return results

def eval(results: List[Result]):
    # scores are negative. So getting 0 is the best score.
    scores = []
    run_time = []
    counter = 0
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append( -1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
                else :
                    counter += 1
            except:
                score -= len(res.actual_ids)
        scores.append(score)
    print(counter)
    return sum(scores) / len(scores), sum(run_time) / len(run_time)

In [3]:
db = VecDBWorst()
records_np = np.random.random((1000000, 70))
records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(records_np)]
_len = len(records_np)
print("start training ...")
db.insert_records(records_dict)
print("start querying ...")
res = run_queries(db, records_np, 10, 10)
print("Evaluation = ",eval(res))

start training ...


KeyboardInterrupt: 